## Loading Previous Functions...

In [1]:
import random

def build_board(row, col, bomb_count=0, non_bomb_character="-"):
    board_temp = ["B"] * bomb_count + [non_bomb_character] * (row*col-bomb_count)

    if bomb_count:
        random.shuffle(board_temp)

    board = []
    for i in range(0, row*col, col):
        board.append(board_temp[i:i+col])
    return board

def display_board(board):
    ## Note bunch of new stuff here. Basically we add some co-ordinates
    ## on the side of the board to help the player figure out their move.
    
    r = len(board)
    c = len(board[0])
    
    s = list(range(0, r))
    s2 = "'{}'  " * c
    
    line = "-" * len(s2)

    for i in range(len(board)):
        print("{}| {}".format(s[i], board[i]))
    print(line)
    print("    " + s2.format(*list(range(0,c)))) ## operator overloading, google "Unpacking"
        
def get_square(x, y, board):
    return board[x][y]

def set_square(x, y, new_val, board):
    board[x][y] = new_val
    
def get3x3(row, col, board):
    offsets = [(-1,-1), (0,-1), (+1,-1),
               (-1, 0), (0, 0), (+1, 0),
               (-1,+1), (0,+1), (+1,+1)]
    
    result = []
    for x, y in offsets:
        if row + x >=0 and col + y >= 0: # This check prevents 'wrapping' arround the board.
            try:
                result.extend(get_square(row + x, col + y, board))
            except IndexError:
                pass
    return result

def set_bomb_count(board):
    r = len(board)
    c = len(board[0])
    
    for row in range(r):
        for col in range(c):
            if get_square(row, col, board) != "B":
                b_count = get3x3(row, col, board).count("B")
                set_square(row, col, str(b_count), board)
    return board

def reveal_square(row, col, player_board, game_board): 
    p_square = get_square(row, col, player_board)
    if p_square in "012345678F":
        pass
    else:
        g_square = get_square(row, col, game_board)
        set_square(row, col, g_square, player_board)  
        if g_square == "B":
            return "GAME OVER"   

def flag_square(row, col, player_board):
    p_square = get_square(row, col, player_board)
    if p_square == "-":
        set_square(row, col, "F", player_board)
    elif p_square == "F":
        set_square(row, col, "-", player_board)

## Won Game Function...

In [2]:
def is_won(p_board, g_board, bombs):

    flags_total = 0
    flags_correct = 0

    for row in range(len(p_board)):
        for col in range(len(p_board[0])):
            p_square = get_square(row, col, p_board)
            g_square = get_square(row, col, g_board)

            if p_square == "F":
                flags_total += 1
                if g_square == "B":
                    flags_correct += 1

    return flags_total - flags_correct == 0 and flags_total == bombs

## Won Game Explanation...

    flags_total = 0
    flags_correct = 0

So our strategy here is to count the number of total flags and subtract from that the number of correct flags. We want this number to be exactly zero *(if you flag every single square on the board of course you are going to flag ever bomb)*.

    return flags_total - flags_correct == 0 and flags_total == bombs

This however, is not a sufficient condition, if someone flags just one bomb correctly then that would be considered a won game. Which is why we check the flags_total is equal to the number of bombs. Moving on...

    for row in range(len(p_board)):
        for col in range(len(p_board[0]))

This for-loop plus inner-for-loop combo we iterate through every single square on the board (via indexing).

    if p_square == "F":
        flags_total += 1

If the p_board has an "F", we increase the total flag_count by one. 

    if g_square == "B":
        flags_correct += 1

And finally, if the square that the player has flagged is in fact a bomb we increment 'flags_correct' by one.

In [3]:
def ui(ROWS, COLS):
    bools = [False, False, False]

    while not bools[0]: 
        x = int(input("Please give me an x value in range {}-to-{}.  ".format(0, ROWS-1)))
    
        if x in range(0, ROWS):
            bools[0] = True
        else:
            print ("Invalid Input, please try again")
              
    while not bools[1]: 
        y = int(input("Please give me an y value in range {}-to-{}.  ".format(0, COLS-1)))
    
        if y in range(0, COLS):
            bools[1] = True   
        else:
            print ("Invalid Input, please try again")
              
    while not bools[2]:
        rf = input('Type "R" to reveal a square, or "F" to flag a square.  ')
              
        if rf in "RrFf":
            bools[2] = True
        else:
            print ("Invalid Input, please try again")
              
    return  x, y, rf
          
def play_game(ROWS, COLS, BOMBS):

    # Setting the game boards...
    g_board = set_bomb_count( build_board(ROWS, COLS, bomb_count = BOMBS) )
    p_board = build_board(ROWS, COLS, bomb_count = 0)
    # display board...
    display_board(p_board)

    # Game Loop...          
    while True:
        x, y, rf = ui(ROWS, COLS)

        if rf in "Rr":
            s = reveal_square(x, y, p_board, g_board)
            if s == "GAME OVER":
                return "YOU LOSE!"
        elif rf in "Ff":
            flag_square(x, y, p_board)
            if is_won(p_board, g_board, BOMBS):
                return "YOU WIN!"
              
        display_board(p_board)

## Explaining the UI function...

I'll be honest, this is some "dodgy" looking code; if I spent more time on it I probably could have come up with something more beautiful than this ugly monstrosity! Notice how repetitive it is? Thats usually a good sign that you can do better. In fact, as my final homework to you go ahead, see if you can improve my function.

Anyway, the basics of this function is that it gets the user to enter valid data with the 'input' built-in. Once we have the data we need we return it. Honestly there is little else to say really. 

Actaully, maybe I should explain while-loops but do I need to? At this point in the guide if you don't know something you should be googling it, not waiting for me to spoon-feed you! 

## Explaining the 'play game' function...

    x, y, rf = ui(ROWS, COLS)
    
So this is a new bit of Python syntax you might need help understanding. The short version is that the 'ui' function returns three values, and these values instead of being returned are being saved as varaibles. Yes you can do that!

    def ui(ROWS, COLS):
    # ...Lots of code...
        return  x, y, rf

In this case the 'x', 'y' and 'rf' values are being saved as 'x', 'y', and 'rf' within the 'play_game' function. I used the same name 'cuz reada-feckin-bility!

## Detour; unpacking.

Here's another example of the above concept:

In [4]:
def simple_example():
    return 1, 2, 3, 4, 5

one, two, three, four, five = simple_example()

print(five, four, three, two, one)

5 4 3 2 1


Note that this only works because we have five names to assign and five items being returned. What happens if we mess up the numbers involved?

In [5]:
one, two = simple_example()

ValueError: too many values to unpack (expected 2)

Yes thats right, we get a rather helpful error message! Python expects the function to return two items but it returns five. And now poor old python doesn't know what to do.

You get it? I bet you don't lol. Here, let me confuse you!

In [ ]:
everything = simple_example()

print(everything)

Thats wierd right? We give Python five names and it works, we give Python two names and it throws and error. And finally, if we give it just one name to assign a value to Python works again. 

The awnser is that when you try to return multiple variables Python returns a tuple containing those variables. 

> What's a tuple? 

Google it. But, if you are simply too lazy to Google just think of a tuples as being a 'specail type' of list. So, you give Python one variable name and Python says:

> "ah I see, you want me to assign the whole 'list' to the name. Okay sure, I can do that."

But when you give Python multiple names to assign values to Python tried to "unpack" the tuple; one name for each item. And thats why you get an error when you give it too many/too few names to assign.

In [ ]:
a, b, c, d, e, f = simple_example() ## To many names to handle!

## Alright, back to the 'play game' function...

    x, y, rf = ui(ROWS, COLS)

        if rf in "Rr":
            s = reveal_square(x, y, p_board, g_board)
            if s == "GAME OVER":
                return "YOU LOSE!"

Alright so what's going on here? well, remember x, y are integers that the player gave us. remember also that rf is one of ["r", "R", "f", "F"].

In our code if the player typed in "R" that means that want to reveal square(x, y). So we set up an if statement to check rf does equal "R". If it does, we call the reveal square function and see what happens. 

In this case the variable name "s" *(a bad variable name, by the way, what a naughty hamster I am!)* equals either nothing at all or the string "GAME OVER". In the latter case we exit the game loop with a return statement. 

     elif rf in "Ff":
            flag_square(x, y, p_board)
            if is_won(p_board, g_board):
                return "YOU WIN!"
                
If rf is an "F" or "f", the player intended to flag a square, and so we do just that. Now that the player have flagged a square we want to check if the game is over (i.e. the player has flagged all the bombs). If they have, we exit the game loop. If not though, we continue...

    display_board(p_board)
    
Prints the board to the console. Oh by the way, I "upgraded" this function since you last saw it, it now not only prints the board but also prints the board cordinates on the side. Cool huh?

But after this statement we reach the end of our function. What happens now? Well, notice all the above code was contained within a while loops:

    While True:
        ## ... main code ...
        
This means that we will continue the above pattern either until the end of time or (more likely) the player wins or loses the game. 

Below I've played a game  to show you how it works. If you wish, you can play a game for yourself too. 

## Playing the Game!

In [6]:
random.seed(20) ## Used for testing... it "rigs" the RNG, lol
play_game(3,3,2) 

0| ['-', '-', '-']
1| ['-', '-', '-']
2| ['-', '-', '-']
------------------
    '0'  '1'  '2'  
Please give me an x value in range 0-to-2.  1
Please give me an y value in range 0-to-2.  1
Type "R" to reveal a square, or "F" to flag a square.  f
0| ['-', '-', '-']
1| ['-', 'F', '-']
2| ['-', '-', '-']
------------------
    '0'  '1'  '2'  
Please give me an x value in range 0-to-2.  1
Please give me an y value in range 0-to-2.  0
Type "R" to reveal a square, or "F" to flag a square.  r
0| ['-', '-', '-']
1| ['2', 'F', '-']
2| ['-', '-', '-']
------------------
    '0'  '1'  '2'  
Please give me an x value in range 0-to-2.  0
Please give me an y value in range 0-to-2.  2
Type "R" to reveal a square, or "F" to flag a square.  r
0| ['-', '-', '1']
1| ['2', 'F', '-']
2| ['-', '-', '-']
------------------
    '0'  '1'  '2'  
Please give me an x value in range 0-to-2.  0
Please give me an y value in range 0-to-2.  1
Type "R" to reveal a square, or "F" to flag a square.  r
0| ['-', '2', '1']
1

'YOU WIN!'

## Conclusion

Aright so after a fair bit of effort we managed to get a fairly basic version of minesweeper working. If you want to take the project further you can always experiment by adding new features or trying to make the game itself more fun to play (is losing on the first turn of the game fun?). But for this guide what we have now will do. 

Alright, thats it folks, peace out!